In [47]:
import pandas as pd
import csv
import time
import datetime as dt
import numpy as np

In [48]:
#read the cycles data
read_data = pd.read_csv('cycles_shpr_29july_sorted.csv', delimiter = ',')


In [49]:
class Bus:
    """
    A class to define how many buses we need
    bus_no defines the number of the bus being used
    list_of_loops provides the index number of the loop
    this bus will travel on
    end_time provides the end time of the travel the bus 
    has done, i.e at what time bus will be free for another loop
    end_stn provides the last station that bus has reached on
    """
    
    def __init__(self, bus_no, end_time, end_stn):
        """
        Initialize the bus_no, its end_time and its end station when you are declaring 
        a new bus is to be added to the system
        a new list, list_of_loops is added here because a list declared with bus_no
        will result in that list being shared with all the class objects, hence we 
        seperaly declare list_of_loops in constructor of the class, which localises it
        """
        #self.end_time = dt.datetime(1900,1,1,0,0,0)
        self.bus_no = bus_no
        self.end_time = end_time
        self.end_stn = end_stn
        self.list_of_loops = []
        
    
    def add_a_loop_to_list_of_loops_of_a_bus(self, loop_no):
        """ To add a new loop to the list of loops already travelled by that bus"""
        self.list_of_loops.append(loop_no)
        

In [50]:
#for converting a string object to datetime object so that it can be used for future refernces
#DO NOT try to convert it into just time by dt.time method, it becomes an object

read_data['arrival_time']= pd.to_datetime(read_data['arrival_time'], format= '%H:%M:%S' )
read_data['departure_time']= (pd.to_datetime(read_data['departure_time'], format= '%H:%M:%S' ))


In [51]:
"""
Declaring a list_of_buses with a list of Bus class objects
All the operations will be performed on this list of buses
"""
list_of_buses = []

#Initialize the list_of_buses with the first route information so that comparisions can be done
list_of_buses.append(Bus(0,read_data.iloc[0,5], read_data.iloc[0,3]))
                     
#adding loop 0 to the list of loops this bus is running on
list_of_buses[-1].add_a_loop_to_list_of_loops_of_a_bus(0)


In [43]:
#Printing out all the buses currently on the route

for bus in list_of_buses:
    print(bus.bus_no,bus.end_time, bus.end_stn)
    for bus_no in bus.list_of_loops:
        print('Loop number=', bus_no)

0 1900-01-01 05:15:00 SHAHAPUR
Loop number= 0


In [52]:
"""
Checkpoint is declared so that we can traverse through 
all the buses to ensure that there is no bus which can 
acomodate the current loop bus_no counts the number of buses
we currently have.

First a single loop is selected(for i in read_data.index), 
then this loop is compared with all the current buses in the system
(for bus in list_of_buses).

If there is a bus which completes its loop before the current bus with 20 
minutes buffer (if(diff_minutes >= 20 and i != 0 and read_data.iloc[i,2] == bus.end_stn))
and is not the first bus and has the same bus station as the end_stn of the bus,
the bus is sent out on this route making the new loop's end time as this bus's end time,
its end station as this bus's end_stn and this loop is added to the list of loops of this
bus. Checkpoint is made 0 to inform that this loop does not need a new bus.

After the (for bus in list_of_buses), we check if the checkpoint is 1
or not. If it is 1, it means that we need to add a new bus to this loop
as no other bus had compatible time for this loop.
"""

checkpoint = 0
#to keep track of number of buses needed
bus_no =0
for i in read_data.index:
    
    for bus in list_of_buses:
        
        #find out the difference in minutes between departure time of loop and bus end time
        diff_minutes = ((read_data.iloc[i,4] - bus.end_time).total_seconds()) / 60.0
        
        """
        if the departure time of this loop is at least 20 minutes more than the end time of 
        the bus and it is not the initial bus and the end station of the bus is same as 
        the arrival loop station of the ith loop 
        """
        if(diff_minutes >= 5 and i != 0 and read_data.iloc[i,2] == bus.end_stn):
            
            #increase bus end time to loop's end time
            bus.end_time = read_data.iloc[i,5]
            bus.add_a_loop_to_list_of_loops_of_a_bus(i)
            bus.end_stn = read_data.iloc[i,3]
            
            #to inform that this loop does not need a new bus
            checkpoint = 0
            #no other buses are checked
            break
            
        #to check if this is not the first route, which has already been allocated a bus
        elif (i == 0):
            #if first loop, skip this one
            continue
        else:
            #informing that this loop might need a new bus
            checkpoint = 1
        
    #no bus was found compatible to this loop, a new bus needs to be added    
    if (checkpoint == 1):
        #increment the bus number to indicate that a new bus is added
        bus_no = bus_no + 1
        list_of_buses.append(Bus(bus_no,read_data.iloc[i,5], read_data.iloc[i,3]))
        list_of_buses[-1].add_a_loop_to_list_of_loops_of_a_bus(i)


In [53]:
for bus in list_of_buses:
    print(bus.bus_no,bus.end_time, bus.end_stn)
    print (bus.list_of_loops)

0 1900-01-01 05:30:00 AASANGAON
[0, 15]
1 1900-01-01 20:30:00 NASHIK
[1, 7, 54, 65, 77, 84, 107, 119, 127]
2 1900-01-01 19:45:00 ASTHA
[2, 117, 142, 178, 205]
3 1900-01-01 23:45:00 AASANGAON
[3, 101, 116, 124, 179, 242]
4 1900-01-01 21:25:00 KINHAVALI
[4, 118, 122, 141, 191, 203]
5 1900-01-01 18:00:00 NASHIK
[5, 106]
6 1900-01-01 23:45:00 KINHAVALI
[6, 108, 168, 212, 241]
7 1900-01-01 20:15:00 KOLIPADA
[8, 97, 125, 143]
8 1900-01-01 19:20:00 BHIVANDI
[9, 113, 128]
9 1900-01-01 21:25:00 KANAVE
[10, 28, 85, 109, 144, 180, 222, 234]
10 1900-01-01 21:05:00 SHAHAPUR
[11, 96, 114, 126, 145, 202, 211, 235]
11 1900-01-01 22:50:00 SHAHAPUR
[12, 50, 94, 146, 182, 198, 223]
12 1900-01-01 21:45:00 CHONDHA
[13, 129, 164, 204, 230]
13 1900-01-01 19:00:00 NASHIK
[14, 78, 120]
14 1900-01-01 21:00:00 TAKIPATHAR
[16, 92, 224, 236]
15 1900-01-01 21:15:00 SHAHAPUR
[17, 57, 103, 156, 206]
16 1900-01-01 20:15:00 MADH
[18, 121]
17 1900-01-01 21:40:00 SHAHAPUR
[19, 86, 98, 115, 137, 157]
18 1900-01-01 20:20:0

In [54]:
print("total buses needed =", len(list_of_buses))

total buses needed = 57
